In [1]:
## Load The Data
import json
import os
#raw = json.load(open('./data/annotations_consistent.json'))

raw = json.load(open('./data/FinEntity.json'))
# raw = data.get("examples")

# for example in raw:
#     for annotation in example['annotations']:
#         #We expect the key of label to be label but the data has tag
#         annotation['label'] = annotation['tag']

In [4]:
pip install transformers


In [18]:
## Preparing Sequence Labeling Data for Transformers
from sequence_aligner.labelset import LabelSet
from sequence_aligner.dataset import TrainingDatasetCRF
from sequence_aligner.containers import TraingingBatch
from transformers import RobertaTokenizerFast


tokenizer = RobertaTokenizerFast.from_pretrained('mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis')
label_set = LabelSet(labels=["Neutral", "Positive", "Negative"])  # label in this dataset
print(label_set.ids_to_label)
print(len(label_set.ids_to_label.values()))
dataset = TrainingDatasetCRF(data=raw, tokenizer=tokenizer, label_set=label_set,tokens_per_batch = 128)
print(len(dataset))

{0: 'O', 1: 'B-Neutral', 2: 'I-Neutral', 3: 'L-Neutral', 4: 'U-Neutral', 5: 'B-Positive', 6: 'I-Positive', 7: 'L-Positive', 8: 'U-Positive', 9: 'B-Negative', 10: 'I-Negative', 11: 'L-Negative', 12: 'U-Negative'}
13
987


In [9]:
from transformers import RobertaTokenizer

# Example input data
raw_data = ["Operating profit totaled EUR 9.4 mn , down from EUR 11.7 mn in 2004 ."]

# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis')

# Tokenize the input data
encodings = tokenizer.encode_plus(raw_data, padding=True, truncation=True, return_tensors='pt')

# Inspect the encodings
print("Tokenized input:", encodings.input_ids)
print("Attention mask:", encodings.attention_mask)


Tokenized input: tensor([[0, 3, 2]])
Attention mask: tensor([[1, 1, 1]])


In [19]:
## Prepare train data and valid data
from torch.utils.data import DataLoader, random_split
import config
train_size = int(config.dev_split_size * len(dataset))
validate_size = len(dataset) - train_size
train_dataset, validate_dataset = random_split(dataset, [train_size, validate_size])

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=TraingingBatch, shuffle=True, )
val_loader = DataLoader(validate_dataset, batch_size=16, collate_fn=TraingingBatch, shuffle=True, )

print(dataset[1].input_ids)
print(dataset[1].labels)
print(dataset[1].attention_masks)
print(tokenizer.decode(dataset[1].input_ids))
print(dataset.label_set.ids_to_label)

[4148, 5, 1313, 526, 6, 16422, 1290, 16, 18142, 231, 207, 71, 10, 8600, 11, 3472, 3365, 8, 9805, 4403, 8003, 6083, 424, 7387, 16, 67, 62, 30, 231, 207, 71, 1963, 1451, 2113, 4, 1437, 1437, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 5, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [20]:
## Bert+Crf
import warnings
from model.DistilBERT_crf import DistilBertCrfForNer
from seqeval import metrics
from transformers import get_linear_schedule_with_warmup
from seqeval.metrics import f1_score, precision_score, accuracy_score
from torch import cuda
import config
from util.train import train_epoch, valid_epoch
from  torch.optim import AdamW

warnings.filterwarnings('ignore')
# 'bert-base-cased' 'yiyanghkust/finbert-pretrain'
label_set = LabelSet(labels=["Neutral", "Positive", "Negative"]) 
model = DistilBertCrfForNer.from_pretrained('mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis', num_labels=len(label_set.ids_to_label.values()))

device = 'cuda:0' if cuda.is_available() else 'cpu'
model.to(device)

len_dataset = len(train_dataset)
t_total = len(train_dataset)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ["bias", "LayerNorm.weight"]
bert_param_optimizer = list(model.bert.named_parameters())
crf_param_optimizer = list(model.crf.named_parameters())
optimizer_grouped_parameters = [
        {'params': [p for n, p in bert_param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay': config.weight_decay, 'lr': config.lr_crf},
        {'params': [p for n, p in bert_param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0,
         'lr': config.lr_crf},

        {'params': [p for n, p in crf_param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay': config.weight_decay, 'lr': config.crf_learning_rate},
        {'params': [p for n, p in crf_param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0,
         'lr': config.crf_learning_rate},

    ]
optimizer = AdamW(optimizer_grouped_parameters, lr=config.lr, eps=1e-6)
warmup_steps = int(t_total * config.warm_up_ratio)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps,
                                                num_training_steps=t_total)

EPOCHS = config.epoch_num # training epoch
for e in range(EPOCHS):
    print("=======START TRAIN EPOCHS %d=======" %(e+1))
    train_loss = train_epoch(e, model, train_loader, optimizer, scheduler,device)
    valid_epoch(e, model, val_loader,device,label_set)



: 

In [14]:
pip install --upgrade transformers



     ---------------------------------------- 0.0/130.7 kB ? eta -:--:--
     --- ------------------------------------ 10.2/130.7 kB ? eta -:--:--
     --- ------------------------------------ 10.2/130.7 kB ? eta -:--:--
     -------- ---------------------------- 30.7/130.7 kB 262.6 kB/s eta 0:00:01
     ----------------------- ------------- 81.9/130.7 kB 459.5 kB/s eta 0:00:01
     --------------------------------- -- 122.9/130.7 kB 654.9 kB/s eta 0:00:01
     --------------------------------- -- 122.9/130.7 kB 654.9 kB/s eta 0:00:01
     --------------------------------- -- 122.9/130.7 kB 654.9 kB/s eta 0:00:01
     --------------------------------- -- 122.9/130.7 kB 654.9 kB/s eta 0:00:01
     --------------------------------- -- 122.9/130.7 kB 654.9 kB/s eta 0:00:01
     --------------------------------- -- 122.9/130.7 kB 654.9 kB/s eta 0:00:01
     --------------------------------- -- 122.9/130.7 kB 654.9 kB/s eta 0:00:01
     --------------------------------- -- 122.9/130.7 kB 6